In this Notebook we implement decision trees for classification of weather. The label is the "relative humadity around 3 pm".
the starting point of this mini project is the EDX course of Python for Data Science by UCSD.

We first follow their approach to picking the tree parameters ad-hoc. Then we consider cross validation to pick the best parameters to avoid overfitting. Once the parameters are finalized we estimate the test performance using Monte Carlo Cross Validation. This is essentially very helpful when the data set is too small that:

A) We may need to use **all** the data for training and nothing is left for out-of sample to estimate the test error.

B) The test data set is too small (or low in label = 1) that any estimate soleyly on that is not reliable (i.e. too volatile) 


In [178]:
# importing packages
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [179]:
data = pd.read_csv('./weather/daily_weather.csv')

In [180]:
data.shape

(1095, 11)

In [181]:
data.columns

Index(['number', 'air_pressure_9am', 'air_temp_9am', 'avg_wind_direction_9am',
       'avg_wind_speed_9am', 'max_wind_direction_9am', 'max_wind_speed_9am',
       'rain_accumulation_9am', 'rain_duration_9am', 'relative_humidity_9am',
       'relative_humidity_3pm'],
      dtype='object')

In [182]:
del data['number']

In [183]:
data.columns

Index(['air_pressure_9am', 'air_temp_9am', 'avg_wind_direction_9am',
       'avg_wind_speed_9am', 'max_wind_direction_9am', 'max_wind_speed_9am',
       'rain_accumulation_9am', 'rain_duration_9am', 'relative_humidity_9am',
       'relative_humidity_3pm'],
      dtype='object')

In [184]:
data.shape

(1095, 10)

In [185]:
# Every column is related to weather around "9am" and the only column with "3pm" is our label.
data.head(5) # Check the range and type of the features. "label" is continuous and needs transformation.

,air_pressure_9am,air_temp_9am,avg_wind_direction_9am,avg_wind_speed_9am,max_wind_direction_9am,max_wind_speed_9am,rain_accumulation_9am,rain_duration_9am,relative_humidity_9am,relative_humidity_3pm
0,918.060000,74.822000,271.100000,2.080354,295.400000,2.863283,0.0,0.0,42.420000,36.160000
1,917.347688,71.403843,101.935179,2.443009,140.471548,3.533324,0.0,0.0,24.328697,19.426597
2,923.040000,60.638000,51.000000,17.067852,63.700000,22.100967,0.0,20.0,8.900000,14.460000
3,920.502751,70.138895,198.832133,4.337363,211.203341,5.190045,0.0,0.0,12.189102,12.742547
4,921.160000,44.294000,277.800000,1.856660,136.500000,2.863283,8.9,14730.0,92.410000,76.740000


In [186]:
data.isnull() #Elementwise True/False
data.isnull().any(axis = 0) # Reports if each column (feature or label) has at least 1 missing value
data.isnull().any(axis = 1) # Reports if at leaset 1 of the columns for each record is missing
data.loc[data.isnull().any(axis = 1),:] # List of all records with at least one missing value
len(data.loc[data.isnull().any(axis =1),:]) # 31 records have missing values
missing_rate = "{0:.2f}".format(len(data.loc[data.isnull().any(axis =1),:])*100 /len(data))
print(missing_rate + "% of the population has missing values.")
print("The number of missing values for each columns are:")
31 - data.loc[data.isnull().any(axis =1),:].count()


2.83% of the population has missing values.
The number of missing values for each columns are:


air_pressure_9am          3
air_temp_9am              5
avg_wind_direction_9am    4
avg_wind_speed_9am        3
max_wind_direction_9am    3
max_wind_speed_9am        4
rain_accumulation_9am     6
rain_duration_9am         3
relative_humidity_9am     0
relative_humidity_3pm     0
dtype: int64

In [187]:
data.dropna(axis = 0,inplace = True)

In [188]:
data.shape # The 31 records were removed "in place"

(1064, 10)

In [189]:
# How to convert the problem to a binary classification.
data.loc[:,'relative_humidity_3pm'].describe()
 


count    1064.000000
mean       35.148381
std        22.365475
min         5.300000
25%        17.360468
50%        24.371286
75%        51.922500
max        92.250000
Name: relative_humidity_3pm, dtype: float64

In [190]:
# To make it a "balanced" data set, we pick 24.37 to be the threshold for conversion to binary
clean_data = data.copy(deep = True)
clean_data.loc[:,'HiHumidLabel'] = 1*(clean_data.loc[:,'relative_humidity_3pm']> 24.37)

In [191]:
clean_data.loc[:,'HiHumidLabel'].mean() # 50%-50% split

0.5

In [192]:
# Store the label in y
y = clean_data.loc[:,['HiHumidLabel']].copy(deep = True)

In [193]:
morning_features = [c for c in clean_data.columns.values if '9am' in c]
morning_features

['air_pressure_9am',
 'air_temp_9am',
 'avg_wind_direction_9am',
 'avg_wind_speed_9am',
 'max_wind_direction_9am',
 'max_wind_speed_9am',
 'rain_accumulation_9am',
 'rain_duration_9am',
 'relative_humidity_9am']

In [194]:
# Store features in X
X = clean_data.loc[:,morning_features].copy(deep = True)

In [195]:
X.head()

,air_pressure_9am,air_temp_9am,avg_wind_direction_9am,avg_wind_speed_9am,max_wind_direction_9am,max_wind_speed_9am,rain_accumulation_9am,rain_duration_9am,relative_humidity_9am
0,918.060000,74.822000,271.100000,2.080354,295.400000,2.863283,0.0,0.0,42.420000
1,917.347688,71.403843,101.935179,2.443009,140.471548,3.533324,0.0,0.0,24.328697
2,923.040000,60.638000,51.000000,17.067852,63.700000,22.100967,0.0,20.0,8.900000
3,920.502751,70.138895,198.832133,4.337363,211.203341,5.190045,0.0,0.0,12.189102
4,921.160000,44.294000,277.800000,1.856660,136.500000,2.863283,8.9,14730.0,92.410000


In [196]:
y.head()

,HiHumidLabel
0,1
1,0
2,0
3,0
4,1


In [197]:
# Split in  Train and Test (Split 2/3 training, 1/3 testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 1234 )

In [198]:
X_train.head()

,air_pressure_9am,air_temp_9am,avg_wind_direction_9am,avg_wind_speed_9am,max_wind_direction_9am,max_wind_speed_9am,rain_accumulation_9am,rain_duration_9am,relative_humidity_9am
149,919.660000,66.920000,105.400000,2.214571,147.000000,3.198824,0.00,0.0,28.140000
327,916.300000,58.496000,175.000000,4.071231,182.400000,4.719943,0.00,0.0,24.050000
347,920.181295,58.908109,57.900885,9.395121,75.724336,11.735431,0.00,0.0,18.933142
243,922.320000,55.526000,215.900000,9.395148,228.500000,10.849159,0.02,60.0,91.120000
8,920.080000,80.582000,40.700000,4.518619,63.000000,5.883152,0.00,0.0,29.580000


In [199]:
print("{0:.2f}".format(y_train.loc[:,'HiHumidLabel'].mean()*100) + "% is the rate of label 1 in Training data set.")
print("{0:.2f}".format(y_test.loc[:,'HiHumidLabel'].mean()*100) + "% is the rate of label 1 in Testing data set.")

49.16% is the rate of label 1 in Training data set.
51.70% is the rate of label 1 in Testing data set.


In [200]:
# To beign with, let's control the number of maximum leaf nodes to be 10
h_classifier_1 = DecisionTreeClassifier(max_leaf_nodes = 10, random_state = 1234)
h_classifier_1.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=10,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1234,
            splitter='best')

In [201]:
prediction = h_classifier_1.predict(X_test)

In [202]:
print("The accuracy is {0:.2f}%".format(accuracy_score(y_true = y_test, y_pred = prediction)*100))

The accuracy is 83.52%


In [203]:
p_test = h_classifier_1.predict_proba(X_test)
p = [p_test[i][1] for i in list(range(len(p_test)))]

In [94]:
print("The Area under the curve is {0:.2f}.".format(roc_auc_score(y_true = y_test, y_score = p)))

The Area under the curve is 0.89.


In [204]:
print(classification_report(y_true = y_test, y_pred = prediction))

              precision    recall  f1-score   support

           0       0.81      0.86      0.83       170
           1       0.86      0.81      0.84       182

   micro avg       0.84      0.84      0.84       352
   macro avg       0.84      0.84      0.84       352
weighted avg       0.84      0.84      0.84       352



In [205]:
pred_all  = h_classifier_1.predict(X)

In [206]:
p_all= h_classifier_1.predict_proba(X)
p = [p_all[i][1] for i in list(range(len(p_all)))]

In [207]:
print("The Area under the curve for the whole data set is {0:.2f}.".format(roc_auc_score(y_true = y, y_score = p)))

The Area under the curve for the whole data set is 0.93.


In [208]:
print("The accuracy for the whole data set is {0:.2f}%".format(accuracy_score(y_true = y, y_pred = pred_all)*100))

The accuracy for the whole data set is 89.19%


# Grid Search Cross Validation
In previous section we picked the complexity of the tree defined by maximum number of leaf nodes of the tree to be 10. The Accuracy and ROC AUC on the test data was **83.52%** and **0.89** respectively. These numbers are **89.19%** and **0.93** on **100%** of the population.

In the following section, we carry out a grid search on two measures of tree complexity; namely
a) Maximum number of leaf nodes
b) Maximum depth of the tree.

We carry out a grid search using cross validation to pick the "best" of these parameters. The best is defined in terms of either average roc auc or average accuracy on the "leaft-out" folds. The number of folds are also varied between 3, 5, and 10 to see how this will impact the optimum parameters.


This method is carried out twice: once the folds are based on split of the **training** data set and once on the plit of **all** the data set. The idea is that if cross validation is done, there is noo point to keep out a portion for **testing**. 

In [213]:
def Implement_GridSearch(X_train, y_train, X_test, y_test, num_fold = 5, scoring = 'accuracy', param = {'max_leaf_nodes': list(range(2,20)), 'max_depth':[None] + list(range(1,6))}, randomseed = 1234):
    dt = DecisionTreeClassifier(random_state = randomseed)
    clf = GridSearchCV(dt, param, scoring = scoring, iid = True, cv = num_fold, refit = True, return_train_score = True )
    clf.fit(X_train, y_train)
    Grid_pred = clf.predict(X_test)
    p_test = clf.predict_proba(X_test)
    p = [p_test[i][1] for i in list(range(len(p_test)))]
    #print(clf.best_params_)
    #print("The accuracy of the grid seach version is {0:.2f}% on test data set.".format(accuracy_score(y_true = y_test, y_pred = Grid_pred)*100))
    #print("The Area under the curve for the grid search is {0:.2f} on test data set.".format(roc_auc_score(y_true = y_test, y_score = p)))
    return clf.best_params_ , accuracy_score(y_true = y_test, y_pred = Grid_pred), roc_auc_score(y_true = y_test, y_score = p) 

In [214]:
folds = [3, 5 , 10]
scoring_method = ['accuracy','roc_auc']

In [215]:
# Train, Test
N_fold = []
SC_method = []
ACC = []
AUC = []
Best_parm = []
Train_X = X_train
Train_y = y_train
Test_X = X_test
Test_y = y_test 
for f in folds:
    for sc in scoring_method:
        bparam, acc, auc = Implement_GridSearch(Train_X, Train_y, Test_X, Test_y, num_fold = f, scoring = sc, param = {'max_leaf_nodes': list(range(2,20)), 'max_depth':[None] + list(range(1,6))}, randomseed = 1234)
        N_fold.append(f)
        SC_method.append(sc)
        Best_parm.append(bparam)
        ACC.append(acc)
        AUC.append(auc)

Results_Train_Test = pd.DataFrame({'Nfold' : N_fold, 'Scoring': SC_method, 'Best': Best_parm, 'Accuracy': ACC, 'AUC' : AUC})
Results_Train_Test

,Nfold,Scoring,Best,Accuracy,AUC
0,3,accuracy,"{'max_depth': 3, 'max_leaf_nodes': 7}",0.840909,0.886732
1,3,roc_auc,"{'max_depth': 3, 'max_leaf_nodes': 12}",0.840909,0.913332
2,5,accuracy,"{'max_depth': 2, 'max_leaf_nodes': 7}",0.846591,0.884567
3,5,roc_auc,"{'max_depth': 2, 'max_leaf_nodes': 8}",0.846591,0.894360
4,10,accuracy,"{'max_depth': None, 'max_leaf_nodes': 9}",0.835227,0.885763
5,10,roc_auc,"{'max_depth': 2, 'max_leaf_nodes': 8}",0.846591,0.894360


In [216]:
# Train, all
N_fold = []
SC_method = []
ACC = []
AUC = []
Best_parm = []
Train_X = X_train
Train_y = y_train
Test_X = X
Test_y = y 
for f in folds:
    for sc in scoring_method:
        bparam, acc, auc = Implement_GridSearch(Train_X, Train_y, Test_X, Test_y, num_fold = f, scoring = sc, param = {'max_leaf_nodes': list(range(2,20)), 'max_depth':[None] + list(range(1,6))}, randomseed = 1234)
        N_fold.append(f)
        SC_method.append(sc)
        Best_parm.append(bparam)
        ACC.append(acc)
        AUC.append(auc)

Results_Train_All = pd.DataFrame({'Nfold' : N_fold, 'Scoring': SC_method, 'Best': Best_parm, 'Accuracy': ACC, 'AUC' : AUC})
Results_Train_All

,Nfold,Scoring,Best,Accuracy,AUC
0,3,accuracy,"{'max_depth': 3, 'max_leaf_nodes': 7}",0.886278,0.928589
1,3,roc_auc,"{'max_depth': 3, 'max_leaf_nodes': 12}",0.889098,0.948997
2,5,accuracy,"{'max_depth': 2, 'max_leaf_nodes': 7}",0.878759,0.928158
3,5,roc_auc,"{'max_depth': 2, 'max_leaf_nodes': 8}",0.878759,0.935935
4,10,accuracy,"{'max_depth': None, 'max_leaf_nodes': 9}",0.890038,0.931458
5,10,roc_auc,"{'max_depth': 2, 'max_leaf_nodes': 8}",0.878759,0.935935


Based on performance on the **test** data set, it seems maximum leaf node of 7 or 8 with max depth of 2 are doing well. Due to slightly better performance of max leaf node of 8, we pick that one. The results based on 100% of the data is also consistent.

There is a sudden improvement on **100%** data when max_depth = None and max_leaf = 9 which is not consistent on **test** data set and for that reason is not considered. Similary for max_depth = 3 and max_leaf_node = 12 both accuracy was higher only in **3-fold** crosss validation when roc_auc was the decision factor. For that reason we ignored that as well.

For the resulting max leaf node = 8 and max_depth = 2; the performance is:

**On testing**: Accuracy: 84.66%, ROC_AUC: 0.89

**On Whole population**: Accuracy: 87.88%, ROC_AUC: 0.93

As pointed out earlier, In case of small data set (not the case here thought), there is no need to keep out a portion of the data out for test and we can repeat all above but using the whole data set.


In [217]:
# All, Test
N_fold = []
SC_method = []
ACC = []
AUC = []
Best_parm = []
Train_X = X
Train_y = y
Test_X = X_test
Test_y = y_test 
for f in folds:
    for sc in scoring_method:
        bparam, acc, auc = Implement_GridSearch(Train_X, Train_y, Test_X, Test_y, num_fold = f, scoring = sc, param = {'max_leaf_nodes': list(range(2,20)), 'max_depth':[None] + list(range(1,6))}, randomseed = 1234)
        N_fold.append(f)
        SC_method.append(sc)
        Best_parm.append(bparam)
        ACC.append(acc)
        AUC.append(auc)

Results_All_Test = pd.DataFrame({'Nfold' : N_fold, 'Scoring': SC_method, 'Best': Best_parm, 'Accuracy': ACC, 'AUC' : AUC})
Results_All_Test

,Nfold,Scoring,Best,Accuracy,AUC
0,3,accuracy,"{'max_depth': None, 'max_leaf_nodes': 7}",0.877841,0.911474
1,3,roc_auc,"{'max_depth': 2, 'max_leaf_nodes': 8}",0.875000,0.931044
2,5,accuracy,"{'max_depth': 2, 'max_leaf_nodes': 6}",0.875000,0.911765
3,5,roc_auc,"{'max_depth': 2, 'max_leaf_nodes': 8}",0.875000,0.931044
4,10,accuracy,"{'max_depth': None, 'max_leaf_nodes': 5}",0.860795,0.907369
5,10,roc_auc,"{'max_depth': 2, 'max_leaf_nodes': 8}",0.875000,0.931044


In [218]:
# All, All
N_fold = []
SC_method = []
ACC = []
AUC = []
Best_parm = []
Train_X = X
Train_y = y
Test_X = X
Test_y = y 
for f in folds:
    for sc in scoring_method:
        bparam, acc, auc = Implement_GridSearch(Train_X, Train_y, Test_X, Test_y, num_fold = f, scoring = sc, param = {'max_leaf_nodes': list(range(2,20)), 'max_depth':[None] + list(range(1,6))}, randomseed = 1234)
        N_fold.append(f)
        SC_method.append(sc)
        Best_parm.append(bparam)
        ACC.append(acc)
        AUC.append(auc)

Results_All_All = pd.DataFrame({'Nfold' : N_fold, 'Scoring': SC_method, 'Best': Best_parm, 'Accuracy': ACC, 'AUC' : AUC})
Results_All_All

,Nfold,Scoring,Best,Accuracy,AUC
0,3,accuracy,"{'max_depth': None, 'max_leaf_nodes': 7}",0.895677,0.934186
1,3,roc_auc,"{'max_depth': 2, 'max_leaf_nodes': 8}",0.890038,0.948372
2,5,accuracy,"{'max_depth': 2, 'max_leaf_nodes': 6}",0.890038,0.933493
3,5,roc_auc,"{'max_depth': 2, 'max_leaf_nodes': 8}",0.890038,0.948372
4,10,accuracy,"{'max_depth': None, 'max_leaf_nodes': 5}",0.884398,0.931334
5,10,roc_auc,"{'max_depth': 2, 'max_leaf_nodes': 8}",0.890038,0.948372


Agian max_depth = 2 and max_leaf_node = 8 is the most common. However, once trained on **all the data set**, the performance changes:

**On testing**: Accuracy: 87.50%, ROC_AUC: 0.93

**On Whole population**: Accuracy: 89%, ROC_AUC: 0.95

# Monte Carlo Cross Validation
So far, we have seen that max_leaf_node = 8 and max_depth = 2 are the best candidates but are we allowed to use all the data to estimate the model? Well, we are using the test data set to have a good estimate of the performance out of time, we could estimate that using Monte Carlo Cross Validation rather than a single data set. Here how it goes:

1) The final model was selected via cross validation with varying folds. 

2) The "test error" is estimated via MCCV

In [222]:
# Number of times
Niter = 1000
AUC = []
ACCURACY = []
for i in list(range(Niter)):
    randomseed = int('12'+ str(i) + '3')
    # Split in  Train and Test (Split 2/3 training, 1/3 testing)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = randomseed)
    # To beign with, let's control the number of maximum leaf nodes to be 10
    h_classifier = DecisionTreeClassifier(max_leaf_nodes = 8, max_depth= 2, random_state = randomseed)
    h_classifier.fit(X_train, y_train)
    prediction = h_classifier.predict(X_test)
    p_test = h_classifier.predict_proba(X_test)
    p = [p_test[i][1] for i in list(range(len(p_test)))]
    AUC.append(roc_auc_score(y_true = y_test, y_score = p))
    ACCURACY.append(accuracy_score(y_true = y_test, y_pred = prediction)*100)
    

In [240]:
AUC = np.array(AUC)
print('The average test AUC is {0:.2f} with standard deviation of {1:.2f}.'.format(AUC.mean() , AUC.std()))
ACCURACY = np.array(ACCURACY)
print('The average test AUC is {0:.2f}% with standard deviation of {1:.2f} percent.'.format(ACCURACY.mean() , ACCURACY.std()))

The average test AUC is 0.93 with standard deviation of 0.01.
The average test AUC is 87.05% with standard deviation of 1.77 percent.


**Further Research:**

A) Change the criterion in the tree from *Gini* to *Enropy* and see how the outcome would change.

B) Change the balance of the classes (by changing the threshold 24.37 early on in the code) to see how the performance varies with imbalanced data. It is expected that MCCV may be more crucial in "rare event" scenario.